In [10]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.pipeline import Pipeline

import plotly.io as pio
from sklearn.impute import SimpleImputer

pio.templates.default = "plotly_white"

%matplotlib inline

#Models for scikit learn


#Model Evaluations
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats

In [17]:
### awaiting dataframe

In [12]:
numeric_features = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak']
categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang']
target_variable = 'num'

In [13]:
X = df[numeric_features + categorical_features]
y = df[target_variable]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [15]:
numeric_features = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [14]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Support Vector Machine': SVC(probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

param_distributions = {
    'Logistic Regression': {
        'classifier__C': stats.loguniform(1e-3, 1e3),
        'classifier__solver': ['liblinear', 'lbfgs']
    },
    'Decision Tree': {
        'classifier__max_depth': [3, 5, 10],
        'classifier__min_samples_split': stats.randint(2, 20)
    },
    'Support Vector Machine': {
        'classifier__C': stats.loguniform(1e-3, 1e3),
        'classifier__kernel': ['linear', 'rbf']
    },
    'K-Nearest Neighbors': {
        'classifier__n_neighbors': stats.randint(3, 20),
        'classifier__weights': ['uniform', 'distance']
    }
}

In [16]:
for model_name, model in models.items():
    print(f"Tuning hyperparameters for {model_name} using RandomizedSearchCV...")
    
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', model)])
    
    random_search = RandomizedSearchCV(
        estimator=clf,
        param_distributions=param_distributions[model_name],
        scoring=make_scorer(roc_auc_score, needs_proba=True),
        n_iter=20,  # 随机选择 20 个参数组合
        cv=5,
        random_state=42
    )
    
    random_search.fit(X_train, y_train)
    
    best_models[model_name] = random_search.best_estimator_
    
    print(f"Best parameters for {model_name}: {random_search.best_params_}")
    print(f"Best cross-validated AUC: {random_search.best_score_:.4f}")
    print("-" * 40)

for model_name, model in best_models.items():
    print(f"Evaluating {model_name} on test set...")
    y_pred = model.predict(X_test)
    
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_pred_proba)
        print(f"Test AUC: {auc:.4f}")
    
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("-" * 40)

ValueError: multi_class must be in ('ovo', 'ovr')